# PTT 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 印出最新文章的「作者」「標題」「時間」
* ② 印出第一頁所有文章的「作者」「標題」「時間」


In [20]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.ptt.cc/bbs/NBA/index.html'
r = requests.get(url)

soup = BeautifulSoup(r.text, "html5lib")

### ② 印出第一頁所有文章的「作者」「標題」「時間」

In [21]:
for d in soup.find_all(class_="title"):
    print(d.text.replace('\t', '').replace('\n', ''))
    try:
        r = BeautifulSoup(requests.get('https://www.ptt.cc'+d.find('a')['href']).text, "html5lib")
        print('作者: ' + r.find(class_='article-meta-value').text)
        print('時間: ' + r.find_all(class_='article-meta-value')[-1].text)
    except:
        continue

[情報] Kenneth Faried 加入拓荒者夏季聯盟隊伍
作者: thnlkj0665 (灰色地帶)
時間: Fri Aug  6 21:02:29 2021
[花邊] Andre Drummond談加入76人的決定及角色
作者: arod1414 (Kawhi Leonard頭號粉絲)
時間: Fri Aug  6 21:12:38 2021
Re: [情報] 記者：Schroder要求一份起薪2500萬美元
作者: laptic (靜夜聖林彼岸花)
時間: Fri Aug  6 21:16:48 2021
[公告] 板規v8.3
[公告] NBA板救濟專案啟動 & 借用板主位置
作者: Vedan (百鬼組-味丹)
時間: Thu Jul 22 00:01:56 2021
[公告] NBA板主徵選(7/23~8/20)
作者: Induction (愛人愛己)
時間: Fri Jul 23 10:37:09 2021
[情報] 2021-22 自由市場 開市總整理   (8/6)
[情報] 2021-22 自由市場 球隊異動表格 (8/6)


In [22]:
for d in soup.find_all(class_="r-ent"):
    print('標題： ', d.find(class_='title').text.replace('\t', '').replace('\n', ''))
    print('作者： ', d.find(class_='author').text.replace('\t', '').replace('\n', ''))
    print('時間： ', d.find(class_='date').text.replace('\t', '').replace('\n', ''))

標題：  [情報] Kenneth Faried 加入拓荒者夏季聯盟隊伍
作者：  thnlkj0665
時間：   8/06
標題：  [花邊] Andre Drummond談加入76人的決定及角色
作者：  arod1414
時間：   8/06
標題：  Re: [情報] 記者：Schroder要求一份起薪2500萬美元
作者：  laptic
時間：   8/06
標題：  [公告] 板規v8.3
作者：  Induction
時間：   6/09
標題：  [公告] NBA板救濟專案啟動 & 借用板主位置
作者：  Vedan
時間：   7/22
標題：  [公告] NBA板主徵選(7/23~8/20)
作者：  Induction
時間：   7/23
標題：  [情報] 2021-22 自由市場 開市總整理   (8/6)
作者：  laigei
時間：   7/28
標題：  [情報] 2021-22 自由市場 球隊異動表格 (8/6)
作者：  laigei
時間：   7/30


### ③ 試著爬爬看其他版的文章

In [23]:
# 改成 Lifeismoney 版，抓出最新一筆的文章

import requests
from bs4 import BeautifulSoup

url = 'https://www.ptt.cc/bbs/Lifeismoney/index.html'
r = requests.get(url)

soup = BeautifulSoup(r.text, "html5lib")

posts = []

for d in soup.find_all(class_="title"):
    try:
        post = {}
        post['標題'] = d.text.replace('\t', '').replace('\n', '')
        
        r = BeautifulSoup(requests.get('https://www.ptt.cc'+d.find('a')['href']).text, "html5lib")
        post['作者'] = r.find(class_='article-meta-value').text
        post['時間'] = r.find_all(class_='article-meta-value')[-1].text
        
        posts.append(post)
    except:
        continue
        
posts = sorted(posts, key= lambda x: x['時間'])
posts[-1]

{'標題': '[情報] 8月份捐血(贈品)資訊 (8/06更新)',
 '作者': 'luxx0171 (聚是偶然、散是必然)',
 '時間': 'Thu Jul 29 22:40:23 2021'}

In [26]:
# 我們發現這個時間好像不是正確的，原因是我們前面存到的時間，其實不是正確的格式

# 改成 Gossiping 版，發生錯誤，因為八卦版會跳轉到一個「十八歲的同意驗證頁面」導致錯誤。
# 參考圖片下載時的解法，加上 cookies 繞過驗證（後面課程會再進行補充）

import requests
from bs4 import BeautifulSoup

url = 'https://www.ptt.cc/bbs/Gossiping/index.html'
r = requests.get(url, cookies={'over18': '1'})

soup = BeautifulSoup(r.text, "html5lib")
soup.title

<title>看板 Gossiping 文章列表 - 批踢踢實業坊</title>

In [32]:
from datetime import datetime

posts = []

for d in soup.find_all(class_="title"):
    try:
        post = {}
        post['標題'] = d.text.replace('\t', '').replace('\n', '')
        
        r = BeautifulSoup(requests.get('https://www.ptt.cc'+d.find('a')['href'], cookies={'over18': '1'}).text, "html5lib")
        post['作者'] = r.find(class_='article-meta-value').text
        post['時間'] = r.find_all(class_='article-meta-value')[-1].text
        
        post['時間'] = datetime.strptime(post['時間'], "%a %b %d %H:%M:%S %Y")
        # 時間轉換：https://stackoverflow.com/questions/10256093/how-to-convert-ctime-to-datetime-in-python
        posts.append(post)
    except:
        continue
        
posts = sorted(posts, key= lambda x: x['時間'])
posts[-1]

{'標題': '[問卦] 外送都要等4、50分是怎麼回事',
 '作者': 'pulato16 (七股哥布林)',
 '時間': datetime.datetime(2021, 8, 6, 21, 50, 7)}